In [0]:
# Framework
# Import dataset
# Define float variables
# Impute missing data
# Prepare date using pipeline
# Perform clustering

In [0]:
# Import data
fileLocation = "dbfs:/FileStore/shared_uploads/akbarazad@outlook.sg/CC_GENERAL.csv"
fileType = "csv"
inferSchema = "false"
firstRowIsHeader = "true"

df = spark.read.format(fileType)\
.option("inferSchema" , inferSchema)\
.option("header" , firstRowIsHeader)\
.load(fileLocation)

In [0]:
# Print metadata
df.printSchema()

root
-- CUST_ID: string (nullable = true)
-- BALANCE: string (nullable = true)
-- BALANCE_FREQUENCY: string (nullable = true)
-- PURCHASES: string (nullable = true)
-- ONEOFF_PURCHASES: string (nullable = true)
-- INSTALLMENTS_PURCHASES: string (nullable = true)
-- CASH_ADVANCE: string (nullable = true)
-- PURCHASES_FREQUENCY: string (nullable = true)
-- ONEOFF_PURCHASES_FREQUENCY: string (nullable = true)
-- PURCHASES_INSTALLMENTS_FREQUENCY: string (nullable = true)
-- CASH_ADVANCE_FREQUENCY: string (nullable = true)
-- CASH_ADVANCE_TRX: string (nullable = true)
-- PURCHASES_TRX: string (nullable = true)
-- CREDIT_LIMIT: string (nullable = true)
-- PAYMENTS: string (nullable = true)
-- MINIMUM_PAYMENTS: string (nullable = true)
-- PRC_FULL_PAYMENT: string (nullable = true)
-- TENURE: string (nullable = true)

In [0]:
# Casting multiple variables
from pyspark.sql.types import *


In [0]:
# Identifying and assigning list of variables
floatVars = list(set(df.columns) - set(['CUST_ID'])) # Remove CUST_ID
for column in floatVars:
  df = df.withColumn(column, df[column].cast(FloatType()))
df.printSchema

Out[5]: <bound method DataFrame.printSchema of DataFrame[CUST_ID: string, BALANCE: float, BALANCE_FREQUENCY: float, PURCHASES: float, ONEOFF_PURCHASES: float, INSTALLMENTS_PURCHASES: float, CASH_ADVANCE: float, PURCHASES_FREQUENCY: float, ONEOFF_PURCHASES_FREQUENCY: float, PURCHASES_INSTALLMENTS_FREQUENCY: float, CASH_ADVANCE_FREQUENCY: float, CASH_ADVANCE_TRX: float, PURCHASES_TRX: float, CREDIT_LIMIT: float, PAYMENTS: float, MINIMUM_PAYMENTS: float, PRC_FULL_PAYMENT: float, TENURE: float]>

In [0]:
df.head()

Out[6]: Row(CUST_ID='C10001', BALANCE=40.90074920654297, BALANCE_FREQUENCY=0.8181819915771484, PURCHASES=95.4000015258789, ONEOFF_PURCHASES=0.0, INSTALLMENTS_PURCHASES=95.4000015258789, CASH_ADVANCE=0.0, PURCHASES_FREQUENCY=0.16666699945926666, ONEOFF_PURCHASES_FREQUENCY=0.0, PURCHASES_INSTALLMENTS_FREQUENCY=0.08333300054073334, CASH_ADVANCE_FREQUENCY=0.0, CASH_ADVANCE_TRX=0.0, PURCHASES_TRX=2.0, CREDIT_LIMIT=1000.0, PAYMENTS=201.8020782470703, MINIMUM_PAYMENTS=139.50978088378906, PRC_FULL_PAYMENT=0.0, TENURE=12.0)

In [0]:
# Imputing data
from pyspark.ml.feature import Imputer

# Identifying variables to perform mean imputation
inputCols = list(set(df.columns) - set(['CUST_ID']))

# Pass parameters to impute function
imputer = Imputer(inputCols = inputCols, outputCols = ["{}_imputed".format(c) for c in inputCols])

# Applying imputation
dfImputed = imputer.fit(df).transform(df) # Returns spark dataframe

# Keep columns with "_imputed"
dfImputed = dfImputed.drop(*inputCols)

# Rename columns
newColumnNames = list(map(lambda x: x.replace("_imputed", ""), dfImputed.columns))
dfImputed = dfImputed.toDF(*newColumnNames)
dfImputed.columns

Out[7]: ['CUST_ID',
 'CASH_ADVANCE',
 'PURCHASES_FREQUENCY',
 'ONEOFF_PURCHASES',
 'PAYMENTS',
 'BALANCE',
 'INSTALLMENTS_PURCHASES',
 'MINIMUM_PAYMENTS',
 'PURCHASES_INSTALLMENTS_FREQUENCY',
 'TENURE',
 'BALANCE_FREQUENCY',
 'PURCHASES_TRX',
 'PURCHASES',
 'CREDIT_LIMIT',
 'PRC_FULL_PAYMENT',
 'ONEOFF_PURCHASES_FREQUENCY',
 'CASH_ADVANCE_TRX',
 'CASH_ADVANCE_FREQUENCY']

In [0]:
# Data preparation
from pyspark.ml.feature import VectorAssembler, Normalizer
from pyspark.ml import Pipeline

# Ignore these variables for modelling
ignore = ['CUST_ID']

# Create vector of all features
assembler = VectorAssembler(inputCols = [x for x in dfImputed.columns if x not in ignore], outputCol = 'features')

# Create normalisation for all features to scale between 0 and 1
normaliser = Normalizer(inputCol = 'features', outputCol = 'normFeatures', p = 1.0) # p refers to power for norm

# Define pipeline
pipeline = Pipeline(stages = [assembler, normaliser])

# Fit the pipeline
transformations = pipeline.fit(dfImputed)

# Apply transformation
dfUpdated = transformations.transform(dfImputed)
dfUpdated.columns

Out[8]: ['CUST_ID',
 'CASH_ADVANCE',
 'PURCHASES_FREQUENCY',
 'ONEOFF_PURCHASES',
 'PAYMENTS',
 'BALANCE',
 'INSTALLMENTS_PURCHASES',
 'MINIMUM_PAYMENTS',
 'PURCHASES_INSTALLMENTS_FREQUENCY',
 'TENURE',
 'BALANCE_FREQUENCY',
 'PURCHASES_TRX',
 'PURCHASES',
 'CREDIT_LIMIT',
 'PRC_FULL_PAYMENT',
 'ONEOFF_PURCHASES_FREQUENCY',
 'CASH_ADVANCE_TRX',
 'CASH_ADVANCE_FREQUENCY',
 'features',
 'normFeatures']

In [0]:
# Building model
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a bisecting k-means model
bkm = BisectingKMeans().setK(2).setSeed(1) # Set number of clusters
model = bkm.fit(dfUpdated.select('normFeatures').withColumnRenamed('normFeatures', 'features')) # normFeatures is dense vector of features

# Make predictions
predictions = model.transform(dfUpdated.select('normFeatures').withColumnRenamed('normFeatures', 'features'))

# Evaluate clustering
evaluator = ClusteringEvaluator()
silhoutte = evaluator.evaluate(predictions)
print(f"Slhoutte with squared euclidean distance: {silhoutte}")

# Show results
print("Cluster centres: ")
centres = model.clusterCenters()
for centre in centres:
  print(centre)

Slhoutte with squared euclidean distance: 0.49855848955650006
Cluster centres: 
[1.56899718e-02 1.07480745e-04 2.60570968e-02 9.19431713e-02
 4.12514135e-02 4.13338382e-02 4.21892243e-02 8.65430186e-05
 2.36687998e-03 1.49477386e-04 1.69572988e-03 6.73444143e-02
 6.69629544e-01 5.20146262e-05 2.03692023e-05 7.78368846e-05
 4.99429055e-06]
[9.87433798e-02 5.52450766e-05 4.51778189e-02 1.56153161e-01
 1.67613049e-01 3.68880540e-02 8.98395022e-02 4.07553710e-05
 1.58891509e-03 1.27041540e-04 1.35422759e-03 8.20154181e-02
 3.19911998e-01 1.37626105e-05 2.00523886e-05 4.36827025e-04
 2.07931012e-05]

In [0]:
# Show cluster assignment
predictions.show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
[0.0,1.0494869036...| 0|
(17,[0,3,4,6,8,9,...| 1|
[0.0,7.8016262066...| 0|
[0.01553169633356...| 0|
[0.0,2.7898494820...| 1|
[0.0,6.5967818061...| 1|
[0.0,2.8618312892...| 1|
[0.0,1.6040674545...| 1|
[0.0,3.0988074386...| 0|
[0.0,1.1113817210...| 0|
[0.0,1.3129624979...| 1|
[0.0,3.8487350134...| 1|
[0.0,8.2693862323...| 1|
[0.0,5.1210854300...| 0|
(17,[0,3,4,6,8,9,...| 1|
[0.09377147681215...| 1|
(17,[0,3,4,6,8,9,...| 1|
[0.0,1.0606699887...| 0|
[0.0,3.6062258948...| 0|
[0.0,4.5616731076...| 1|
+--------------------+----------+
only showing top 20 rows